# BAYESIAN OPTIMIZATION - CODING

![BO](BO.jpg)

## PSEUDO CODE

This script aims to realize Bayesian Optimization using sequential Gaussian Process (GP) as surrogate model. The algorithm is based on the following steps:

1. **Initialization**: Select a set of initial points to evaluate the objective function.
2. **Model Fitting**: Fit a GP model to the data.
3. **Acquisition Function**: Define an acquisition function to determine the next point to evaluate.
4. **Optimization**: Find the next point that maximizes the acquisition function.
5. **Update**: Evaluate the objective function at the selected point and add it to the dataset.
6. **Repeat**: Repeat steps 2-5 until the stopping criterion is met.

![BO](BAYESIAN-OPTIMIZATION.png)

## INGREDIENTS

0: THE SEARCH SPACE $X$ - ${[lb_1,ub_1],...,[lb_m,ub_m]}$

1: THE OBJECTIVE FUNCTION $f(x)$ - the black box

2: THE DATASET $D$ - $\{(x_1, y_1), (x_2, y_2), ..., (x_t, y_t)\}$

3: THE SURROGATE MODEL $M(x)$ - Gaussian Process

4: THE ACQUISITION FUNCTION $S(x)$ - Expected Improvement (EI)

5: THE OPTIMIZATION ALGORITHM - L-BFGS-B

6: THE STOPPING CRITERION - Maximum number of iterations / Tolerance

## REALIZATION

### Helpers

In [14]:
# DEPENDENCIES
import numpy as np
import matplotlib.pyplot as plt
# use pop-up window for plots
%matplotlib qt5 
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation


In [15]:
# CONFIGURABLE PARAMETERS

## Hyperparameters
noise_lvl = 0.1
k_alpha = 1
k_lambda = 1

## Process Control
num_init_points = 10
num_iter = 20
tolerance = 1e-3
gap = np.inf

## Acquisition Function - UCB
beta = 2

In [16]:
## Search Space
### parameter 1 - a
lb_a = -6
ub_a = 4
c_a = (ub_a + lb_a) / 2
print(c_a)

### parameter 2 - b
lb_b = -4
ub_b = 6
c_b = (ub_b + lb_b) / 2
print(c_b)

-1.0
1.0


In [17]:
# THE OBJECTIVE FUNCTION - this is actually the black box, here I use a simple example to represent the black box
def objective_function(a, b):
    ret = -1*(a-c_a)**2 - 1*(b-c_b)**2 + 25
    # noise term
    noise = np.random.normal(0, noise_lvl)
    ret += noise
    return ret

In [18]:
# THE SURROGATE MODEL - Gaussian Process - Kernel Function
## Matern 2.5 as the kernel function
def kernel(p1, p2, matern_alpha = 1, matern_lambda = 1):
    # define the distance between two points p1 and p2
    d= np.linalg.norm(p1 - p2)
    k = matern_alpha * (1 + np.sqrt(5) * d / matern_lambda + 5 * d**2 / (3 * matern_lambda**2)) * np.exp(-np.sqrt(5) * d / matern_lambda)
    return k

## Mean and Variance of the Gaussian Process
def mean_var(x, D, K_alpha, k_lambda):
    # assume x is a variable of 1x1
    # calculate the mean and variance of the Gaussian Process
    x_dim = x.shape[1] # assume each colum is a dimension of x
    num_D = D.shape[0] # number of data points in D
    
    Ktt = np.zeros((num_D, num_D))
    for i in range(num_D):
        for j in range(num_D):
            Ktt[i, j] = kernel(D[i,:-1], D[j,:-1], K_alpha, k_lambda)
    Kttn = Ktt + np.eye(num_D) * noise_lvl
    IKttn = np.linalg.inv(Kttn)
    
    Kpt = np.zeros((1, num_D))
    for i in range(num_D):
        Kpt[0, i] = kernel(x, D[i,:-1], K_alpha, k_lambda)
    Ktp = Kpt.T
    Kpp = kernel(x, x, K_alpha, k_lambda)
    
    y = D[:,-1].reshape(-1,1)
    
    mean = Kpt @ IKttn @ y
    var = Kpp - Kpt @ IKttn @ Ktp
        
    return mean, var

In [19]:
# THE ACQUISITION FUNCTION - UPPER CONFIDENCE BOUND
def S(x, D):
    mean, var = mean_var(x, D, k_alpha, k_lambda)
    s = mean + beta * np.sqrt(var)
    return s

In [20]:
# THE OPTIMIZATION FUNCTION FOR THE ACQUISITION FUNCTION
def optimize_acquisition(D):
    # randomly search for the point with the highest acquisition function value
    num_search = 100
    max_s = -np.inf
    max_x = None
    cur_x = np.zeros((1,2))
    for i in range(num_search):
        cur_x[0,0] = np.random.uniform(lb_a, ub_a)
        cur_x[0,1] = np.random.uniform(lb_b, ub_b)
        cur_s = S(cur_x, D)
        if cur_s > max_s:
            max_s = cur_s
            max_x = cur_x.copy()
    return max_x

### The Optimization Process

In [21]:
# PHASE 1 - INITIAL DATASET
D = np.zeros((num_init_points, 3)) # first column is a, second column is b, third column is the objective function value
for i in range(num_init_points):
    a = np.random.uniform(lb_a, ub_a)
    b = np.random.uniform(lb_b, ub_b)
    D[i, 0] = a
    D[i, 1] = b
    D[i, 2] = objective_function(a, b)

In [22]:
# # plot 3D scatter plot
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(D[:, 0], D[:, 1], D[:, 2])
# ax.set_xlabel('a')
# ax.set_ylabel('b')
# ax.set_zlabel('objective function value')
# ax.set_xlim(1.1*lb_a, 1.1*ub_a)
# ax.set_ylim(1.1*lb_b, 1.1*ub_b)
# plt.show()

In [23]:
# PHASE 2 - OPTIMIZATION ITERATIONS
for i in range(num_iter):
    
    # (1) use the surrogate model to approach the posterior distribution of the objective function, modeled by Gaussian Process which can be characterized by the mean function and the covariance function
    
    ## mean, var = mean_var(x, D, 1, 1)
    
    # (2) determine the next point to evaluate using the acquisition function S(D)
    x_eval = optimize_acquisition(D)
    
    # (3) evaluate the objective function at the next point
    f_eval = objective_function(x_eval[0,0], x_eval[0,1])
    
    # (4) update the dataset D
    newpoint = np.zeros((1,3))
    newpoint[0,0] = x_eval[0,0]
    newpoint[0,1] = x_eval[0,1]
    newpoint[0,2] = f_eval
    D = np.vstack((D, newpoint))
    
    # (5) check the stopping criterion
    gap = np.abs(D[-1, -1] - D[-2, -1])
    if gap < tolerance:
        break
    


In [24]:
# number of points in D
print(D.shape[0])

30


In [25]:
# VISUALIZATION
# plot the final dataset in animation, mark the order of the points
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('a')
ax.set_ylabel('b')
ax.set_zlabel('objective function value')
ax.set_xlim(1.1*lb_a, 1.1*ub_a)
ax.set_ylim(1.1*lb_b, 1.1*ub_b)
ax.set_zlim(-100, 10)

def update(i):
    ax.scatter(D[:i, 0], D[:i, 1], D[:i, 2], c='r')
    ax.text(D[i, 0], D[i, 1], D[i, 2], str(i), color='black')
    return ax

ani = FuncAnimation(fig, update, frames=D.shape[0], repeat=False)

plt.show()

In [26]:
# print the final value
print(D[-1, -1])

# print the max value
print(np.max(D[:, -1]))

# print the argmax
print(D[np.argmax(D[:, -1]), :-1])

22.49653264774721
22.69702068765539
[0.42340188 0.45775822]
